# Big Data Processing

## Distributed Computing

As we know from [Chapter 1](01_Introduction), Big Data is data whose volume, velocity, and variety requires *innovative forms of information processing*. In this chapter, we want to discuss in greater detail why this is the case and how Big Data can processed. 

The foundation of any large computational effort is *parallelism*. There is a famous quote from computer science pioneer Grace Hopper: "*In pioneer days they used oxen for heavy pulling, and when one ox couldn't budge a log, they didn't try to grow a larger ox. We should be trying for bigger computers, but for more systems of computers*. In other words, large tasks can only be solved by pooling resources. There are three general methods for the parallelization of computational tasks. 

### Parallel Programming Models

First approach *message passing* tasks are executed independently in isolated environments. Whenever these tasks want to communicate, they send messages to each other. This way, the tasks can exchange data between each other, e.g., because the data is required by different parts of the computation. This communication can be done locally on one physical machine, by using the provided functions by the operating system, or remotely in a distributed environment by communicating via the network. 

The second approach is *shared memory*. In this case, the computational tasks are not performed in isolated environments, but share a common address space in the memory, i.e., they can read and write the same variables. Interactions between the tasks happens by updating the values of variables in the shared memory. Sharing memory within a single physical machine is directly supported by the operating system, and may even be a property of the model for parallelization (threads share the same memory, processes not). Sharing memory across different physical machines is also possible, e.g., via network attached storage of other networking solutions, but usually has some communication overhead. 

The third approach is *data parallelism*. Similar to message passing, tasks are executed independently in isolated environments. The difference to message passing is that the tasks do not need to communicate with each other, because the solution of the computational tasks does not require intermediary results of other tasks. Thus, the application of data parallelism is limited to problems where this strong decoupling of tasks is possible. Such problems are also called *embarrassingly parallel*. 

### Distributed Computing for Data Analysis

Since Big Data is to large to compute or store on single physical machines, we need a distributed environment for computations that involve Big Data. Before computational centers started to account for Big Data, the architecture of such a *compute cluster* was similar to the outline below. 

<img src="images/computing_architectures.png" alt="Distributed Computing Outline" style="width: 600px;"/>

There is a layer for data storage and a layer for computations. Both are using different *nodes* in the compute cluster. Each node is a physical machine. Data storage nodes must provide fast storage (latency, throughput, or both), but do not require much computational power. This is usually implement in a database or a *storage area network* (SAN). Vice versa, compute nodes must provide the computational power through CPUs (and possibly GPUs) and a sufficient amount of memory, local storage is less important and often only used for caching and the installation of software. A user of such a system submits jobs to a job queue to gain insights. For the analysis of data, this means that the data is stored in the database or SAN and then accessed by the compute nodes to generate the desired results of the analysis, from which the data scientists can get insights. 

All three parallelization modes we discussed above can be implemented in such a traditional distributed compute cluster. However, none of these approaches is suitable for big data applications in such a compute cluster. Message passing and shared memory have the biggest scalability problems. 

<img src="images/mpi_sm.png" alt="Distributed Computing with MPI/SM" style="width: 600px;"/>

Since it is unclear which parts of the data are required by the different parallel tasks, it is possibly that every compute node must load all data. While this is not a problem for small data sets, this does not scale with large data sets. Imagine that Terabytes, or even Petabytes of data would have to be copied regularly over the network. The transfer of the data would block the execution of the analysis and the compute nodes would be mostly idle, waiting for data. This does not even account for additional network traffic due to the communication between the tasks. 

Data parallelization fares a bit better, but also does not scale. 

<img src="images/data_parallelism.png" alt="Distributed Computing with Data Parallelism" style="width: 600px;"/>

The advantage of message passing and shared memory is that only parts of the data must be copied to each compute node. While this decreases the stress on the network, all data must still be transfered over network. Thus, data parallelization can handle larger amounts of data than message passing and shared memory, at some point the amount of data becomes to large for the transfer via the network. 

### Data Locality

We see that there is a fundamental problem with traditional distributed computing for big data, which is why we need the *innovative forms of information processing*. The solution is actually quite simple: if the problem is that we cannot copy our data over the network, we must change our architecture such that avoid that. The straightforward way to achieve this is to break the separation of the storage layer from the compute layer: all nodes both store data and can perform computations on that data. 

<img src="images/data_locality.png" alt="Distributed Computing with Data Locality" style="width: 400px;"/>

In the following, we explain how this is implemented in practice. We discuss the MapReduce programming model that became the defacto standard for Big Data applications. Then, we show Apache Hadoop and Apache Spark to demonstrate how the distributed computing with Big Data is implemented.

## MapReduce

The MapReduce paradigm for data the data parallelization to enable BigData processing was [published by Google in 2004](https://doi.org/10.1145/1327452.1327492). The general idea is to describe computations using two different kinds of functions: *map* functions and *reduce* functions. Both functions work with *key-value pair*. Map functions implement the embarrassingly parallel part of algorithms, reduce functions aggregate the results. The concept of map functions and reduce functions is not unique to MapReduce, but a general concept that can be found in many functional programming languages. To enable Big Data, MapReduce introduces a third function, the *shuffle*. The only task of the *shuffle* is to arrange intermediate results, i.e., to facilitate the communication between the map and reduce functions. The following figure gives an overview of the dataflow of MapReduce. 

<img src="images/map_reduce_complete.png" alt="Outline of MapReduce" style="width: 600px;"/>

### map()

The map function gets initial key-value pairs. These are, e.g., read from the the distributed storage or the result of a prior computation using MapReduce. The map function than performs a computation on a *single* key-value pair and stores the results in new key-value pairs. Because the map function only gets a single key-value pair as input, data parallelization is trivial: theoretically, the map function could run in parallel for all key-value pairs without any problem. The map function is defined as

$$map(f_{map}, <key1, value1>) \rightarrow list(<key2, value2>)$$

where $f_{map}$ is a *user-defined function* (UDF) defined by the user of the MapReduce framework. The UDF defines the computation, i.e., how the input key-value pair is transformed into the list of output key-value pairs. Depending on the UDF $f_{map}$, the input keys and output keys could be same or different. While the general concept of MapReduce does not have any restrictions on the type and values of the keys, implementations of MapReduce may restrict this. For example, in the initial implementation of MapReduce by Google, all keys and values were strings and users of MapReduce were expected to convert the types within the map and reduce functions, if required.

### shuffle()

The key-value pairs computed by the map function are organized by the shuffle function, such that the data is grouped by the key. These are then organized by the shuffle and grouped by their keys. Thus, we have

$$shuffle(list<key2, value2>) \rightarrow list(<key2, list(value2)>),$$

i.e., a list of values per key. Often, these data from shuffle is sorted by key, because this can sometimes improve the efficiency of subsequent tasks. The shuffling is often invisible to the user performed in the background by the MapReduce framework. 

### reduce()

The reduce function operates on all values for a given key and aggregates the data into a single result per key.  The reduce function is defined as

$$reduce(f_{reduce}, <key2, list(value2)>) \rightarrow value3$$

where $f_{reduce}$ is a UDF. The UDF $f_{reduce}$ performs the reduction to a single value for one key and gets as input the key and the related list of values. Similar as for the map function, there is no restriction on the type or the values that are generated. Depending on the task, the output could, e.g.,  be key value pairs, integers, or textual data. 

### Word Count with MapReduce

The concept of MapReduce is relatively abstract, unless you are used to functional programming. How MapReduce works becomes clearer with an example. The "Hello World" of MapReduce is the word count, i.e., using MapReduce to count how often each word occurs in a text. This example is both practically relevant, e.g., to create a bag-of-words, and well suited to demonstrate how MapReduce works. 

We use the following text as example: 

```
What is your name?
The name is Bond, James Bond.
```

Our data is stored in a text file with one line per sentence. Our initial keys are the line numbers, our initial values the text in the lines. Thus, we start with these key-value pairs.

```
<line1, "What is your name?">
<line2, "The name is Bond, James Bond.">
```

The map function is defined such that it emits the pair <word, 1> for each word in the input. When we apply this to our input, we get the following list of key-value pairs.

```
<"what", 1>
<"is", 1>
<"your", 1>
<"name", 1>
<"the", 1>
<"name", 1>
<"is", 1>
<"bond", 1>
<"james", 1>
<"bond", 1>
```

The shuffle then groups the values by their keys, such that all values for the same key are in a list.
```
<"bond", list(1, 1)>
<"is", list(1, 1)>
<"james", list(1)>
<"name", list(1, 1)>
<"the", list(1)>
<"what", list(1)>
<"your", list(1)>
```

As reduce function, we output one line for each key. The lines contain the current key and the sum of the values of that key.

```
bond 2
is 1
james 1
name 2
the 1
what 1
your 1
```

### Parallelization

The design of MapReduce enables parallelization for every step of the computational process. The input can be read in chunks to parallelize the creation of the initial key-value pairs. For example, we could have multiple text files, each with 1000 lines that could be processed in parallel. The parallelism is limited by the throughput of the storage and makes more sense, if the data is distributed across multiple physical machines. 

map() can be applied to each key-value pair independently and the potential for parallelism is only limited by the amount of data. 

shuffle() can start as soon as the first key-value pair is processed by the map function. This reduces the waiting times, such that the shuffling is often finished directly after the last computations for map are finished.

reduce() can run in parallel for different keys. Thus, the parallelism is only limited by the number of unique keys created by map(). Moreover, reduce() can already start, once all results for a key are available. This is where sorting by shuffle can help. If the results passed to reduce() are sorted, reduce can start processing for a key, once it sees results for the next key. 

## Apache Hadoop

## Apache Spark